In [ ]:
import sys
import math
import random
import numpy as np

def generar_Ts(lambda_max):
    return - (1/lambda_max) * math.log(random.random())


def simulation(cant_servrs, Tn, lambda_a, lambda_max):
    #Inf simu
    inf = random.randint(0,sys.maxsize)
    
    #Inicializando variables en 0
    t = 0
    Na = 0
    n = 0 
    #Inicializando variables en 0 dependiendo de la cantidad de servidores
    C = [0] * cant_servrs 
    n_i = [0] * cant_servrs 
    
    #Inicializando tiempos
    T = [generar_Ts(lambda_a)]  
    t_a = T[0]  
    t_d = [inf] * cant_servrs 
    
    #Inicializando listas de tiempos
    Arrivals = []
    Departures = []
    Running = []  
    Busy = np.zeros(cant_servrs)

    while True:
        # cases
        if t_a == min([t_a] + t_d) and t_a < Tn: 
            t = t_a
            Na = Na + 1
            to = t + generar_Ts(lambda_a)
            T.append(to)
            t_a = to
            Arrivals.append(t)
            if n < cant_servrs:
                for i in range(cant_servrs):
                    if n_i[i] == 0:  # is empty
                        Running.append(t)
                        n_i[i] = Na
                        n = n + 1
                        Y = generar_Ts(lambda_max)
                        t_d[i] = t + Y
                        Busy[i] += t_d[i]-t
                        #break
            else:
                n = n + 1
                
        elif min(t_d) == min([t_a] + t_d) and min([t_a] + t_d) < Tn:
            ti = np.argmin(t_d)
            index = t_d.index(min(t_d))
            t = ti
            C[index] = C[index] + 1
            Departures.append(t)
            n = n - 1
            if n < cant_servrs:
                n_i[index] = 0
                t_d[index] = inf
            else:
                Running.append(t)
                m = max(n_i)
                n_i[index] = m + 1
                Y = generar_Ts(lambda_max)
                t_d[index] = t + Y

        elif min([t_a] + t_d) > Tn and n > 0:
            temp = np.argmin(t_d)
            t = t_d[temp]
            n -= 1
            C[temp] = C[temp] + 1
            Departures.append(t)
            if n < cant_servrs:
                n_i[temp] = 0
                t_d[temp] = inf
            else:
                index = max(n_i) + 1
                #Running.append(t - Arrivals[index])
                Running.append(t)
                Y = generar_Ts(lambda_max)
                n_i[temp] = index
                t_d[temp] = t + Y
                Busy[temp] += t_d[temp]-t

        elif min([t_a] + t_d) > Tn and n == 0:
            t = t + abs(t - Departures[-1])
            Tp = max(t - Tn, 0)
            return Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i

def output_task1(cant_cant_servrs, Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i):
    print("Solicitudes atendidas por servidor: ",  C)
    print("Solicitudes atendidas totales: ",  Na)
    print("Tiempo que cada servidor estuvo ocupado: ", Busy) 
    print("Tiempo que cada servidor estuvo desocupado: " , np.maximum(np.ones(cant_cant_servrs)*t - Busy,0))
    print("Tiempo total de solicitudes en cola: " , np.round(np.sum(Running), 5))
    print("Tiempo promedio de cada solicitud en cola: " , np.round(np.mean(Running), 5))
    print("El promedio en que las solicitudes estuvieron en cola cada segundo es de: " , np.round(np.mean(Running)*60, 5))
    print("El momento de salida de la ultima solicitud es: " , Departures[-1])
    print("\n")

In [1]:
import sys
import math
import random
import numpy as np

def generar_Ts2(t, lambda_max):
    return t - (1/lambda_max) * math.log(random.random())


def simulation2(cant_servrs, Tn, lambda_max, lambda_b):
    #Inf simu
    inf = random.randint(0,sys.maxsize)
    
    #Inicializando variables en 0
    t = 0
    ti = 0 
    
    Na = 0
    n = 0 
    #Inicializando variables en 0 dependiendo de la cantidad de servidores
    C = [0] * cant_servrs 
    n_i = [0] * cant_servrs 
    
    #Inicializando tiempos
    T = generar_Ts2(t, lambda_max) 
    t_a = T  
    t_d = [inf] * cant_servrs 
    
    #Inicializando listas de tiempos
    Arrivals = []
    Departures = []
    Waiting = []
    Busy = np.zeros(cant_servrs)

    while (n < Tn or n == 0) :
        # cases
        if t_a == min(min(t_d), t_a): 
            if t < Tn:
                t = t_a
                Na = Na + 1
                to = generar_Ts2(t, lambda_max)
                t_a = to
                Arrivals.append(t)
                if n < cant_servrs:
                    for i in range(cant_servrs):
                        if n_i[i] == 0:  # is empty
                            n_i[i] = Na
                            Y = generar_Ts2(t, lambda_b)
                            t_d[i] = Y
                            Busy[i] += t_d[i]-t
                            break
            else:
                ti = ti + 1
                Waiting.append(min(t_d) - t)
            n = n + 1
                
        else: 
            ti = np.argmin(t_d)
            index = t_d.index(min(t_d))
            t = ti
            C[index] += 1
            Departures.append(t)
            if n < cant_servrs:
                n_i[index] = 0
                t_d[index] = inf
            else:
                n_i[index] = Na + 1
                Y = generar_Ts2(t, lambda_b)
                t_d[index] = Y
                Busy[index] += t_d[index]-t
            n -= 1
            
    return Arrivals, Departures, Busy, Waiting, C, Na, t, ti, n_i

def output_task(servers, Arrivals, Departures, Busy, Waiting, C, Na, t, ti, n_i, t_sim):
    print("Total de solicitudes ",  Na)
    print("Cada servidor atendió: ",  C)
    print("Tiempo que cada servidor estuvo ocupado: ", Busy) 
    print("Tiempo que cada servidor estuvo desocupado: " , t_sim - Busy)
    print("Tiempo total de solicitudes en cola: " , np.round(np.sum(Waiting), 5))
    print("Tiempo promedio de cada solicitud en cola: " , np.sum(Waiting)/servers)
    print("Cantidad de procesos en espera: " , ti)
    print("El momento de salida de la ultima solicitud es: " , t_sim - Departures[-1])
    print("\n")

c:\users\dell\appdata\local\programs\python\python37-32\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\dell\appdata\local\programs\python\python37-32\lib\site-packages\numpy\.libs\libopenblas.FN5FF57TWHUYLRG54LA6B33EZPHYZZL4.gfortran-win32.dll
c:\users\dell\appdata\local\programs\python\python37-32\lib\site-packages\numpy\.libs\libopenblas.JKAMQ5EVHIVCPXP2XZJB2RQPIN47S32M.gfortran-win32.dll
  stacklevel=1)


# <font color=red>Task 1 </font>

In [ ]:
#PRIMER CODIGO
print("\nProveedor 1 – Gorilla Megacomputing: \n")
Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i = simulation(1, 3600, 40, 100)
output_task1(1, Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i)

print("\nProveedor 2 – Ants smart computing: \n")
Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i = simulation(10, 3600, 40, 10)
output_task1(10, Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i)

In [ ]:
#SEGUNDO CODIGO
print("\nProveedor 1 – Gorilla Megacomputing: \n")
Arrivals, Departures, Busy, Waiting, C, Na, t, ti, n_i= simulation2(1, 3600, 40, 100)
output_task(1, Arrivals, Departures, Busy, Waiting, C, Na, t, ti, n_i, 3600)

print("\nProveedor 2 – Ants smart computing: \n")
Arrivals, Departures, Busy, Waiting, C, Na, t, ti, n_i= simulation2(10, 3600, 40, 10)
output_task(10, Arrivals, Departures, Busy, Waiting, C, Na, t, ti, n_i, 3600)


Proveedor 1 – Gorilla Megacomputing: 



# <font color=red>Task 2 </font>

Empíricamente la cantidad de servidores que se necesitan alquilar para **Ants smart computing** son **20**. 
Esto debido a que es la cantidad que al realizar probarlo repetidas veces, maximiza el tiempo de ocupados y minimiza los servidores desocupados. 

# <font color=red>Task 3 y 4 </font>

Teniendo 6000 solicitudes por minuto: 

In [ ]:
print("\nProveedor 1 – Gorilla Megacomputing: \n")
Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i = simulation(1, 3600, 6000/60, 100)
output_task1(1, Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i)

print("\nProveedor 2 – Ants smart computing: \n")
Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i = simulation(10, 3600, 6000/60, 10)
output_task1(10, Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i)

# <font color=red>Task 5 </font>

**RECOMENDACIONES**

- Se le recomienda a la junta ejecutiva no utilizar demasiados servidores ya que la cantidad de servidores no implica rendimiento. Por lo tanto, en las pruebas se estimó que 20 servidores puede servir como estimador de la cantidad de servidores que maximiza la ejecución. 

In [ ]:
#Pruebas 

print("\nProveedor 2 – Ants smart computing: 15 servidores \n")
Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i = simulation(25, 3600, 2400/60, 10)
output_task1(25, Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i)

print("\nProveedor 2 – Ants smart computing: 13 servidores \n")
Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i = simulation(19, 3600, 2400/60, 10)
output_task1(19, Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i)

print("\nProveedor 2 – Ants smart computing: 19 servidores \n")
Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i = simulation(19, 3600, 2400/60, 10)
output_task1(19, Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i)

print("\nProveedor 2 – Ants smart computing: 20 servidores \n")
Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i = simulation(20, 3600, 2400/60, 10)
output_task1(20, Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i)

print("\nProveedor 2 – Ants smart computing: 21 servidores \n")
Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i = simulation(20, 3600, 2400/60, 10)
output_task1(20, Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i)

print("\nProveedor 2 – Ants smart computing: 25 servidores \n")
Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i = simulation(25, 3600, 2400/60, 10)
output_task1(25, Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i)

In [ ]:
#PRUEBA CON SAMUEL

#1
print("\nProveedor 2 – Ants smart computing: 10 servidores \n")
Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i = simulation(10, 40, 40, 10)
output_task1(10, Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i)

#2
print("\nProveedor 2 – Ants smart computing: 10 servidores \n")
Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i = simulation2(10, 40, 40, 10)
output_task(10, Arrivals, Departures, Busy, Tp, C, Running, Na, t, n_i, 40)